## Demo

In [1]:
from models import OOIDataset

In [2]:
# This will load the dataset by reading through the zarr meta and load as dask arrays
ooi_ds = OOIDataset(dataset_id="RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample")

In [3]:
ooi_ds

<RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample: 543.6 GB>
Dimensions: (time)
Data variables: 
    bottom_pressure
    bottom_pressure_qc_executed
    bottom_pressure_qc_results
    deployment
    driver_timestamp
    ingestion_timestamp
    internal_timestamp
    port_timestamp
    press_trans_temp
    time

In [4]:
start_idx = ooi_ds.time.data.chunksize[0] * 401

In [40]:
# Grab the dimension array
# As of 6/14/2023, @lsetiawan and @dcherian found that there are some issues
# in the dataset, so we'll just grab half the data,
# seems like there are some repetitions that happened after chunk 401
time = ooi_ds.time[start_idx:]

In [41]:
time

<xarray.DataArray 'time' (time: 5252101366)>
dask.array<getitem, shape=(5252101366,), dtype=float64, chunksize=(12000000,), chunktype=numpy.ndarray>
Dimensions without coordinates: time
Attributes:
    axis:           T
    calendar:       gregorian
    long_name:      time
    standard_name:  time
    units:          seconds since 1900-01-01 0:0:0

In [7]:
data_vars = {}
for k, v in ooi_ds.variables.items():
    if k not in ooi_ds.dimensions:
        data_vars[k] = v[start_idx:]

## Create dataset 

Now try to create dataset here and the index

In [155]:
import xarray as xr
from xarray.core.indexes import Index, PandasIndex
from typing import Mapping, Any, Hashable
from xarray.core.variable import Variable
from dask.utils import memory_repr
import numpy as np
import dask
import pandas as pd

In [12]:
import numpy as np
import dask

In [13]:
from dask.distributed import Client

In [14]:
client = Client()

In [15]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 10,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:65143,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:65158,Total threads: 2
Dashboard: http://127.0.0.1:65163/status,Memory: 6.40 GiB
Nanny: tcp://127.0.0.1:65146,


In [168]:
class DaskIndex(Index):
    
    def __init__(self, array: Any, dim: Hashable, coord_dtype: Any = None, attrs = None):
        self.attrs = attrs
        # Create lookup table
        index = self._create_lookup(array)

        if index.name is None:
            index.name = dim

        self.index = index
        self.dim = dim

        self.coord_dtype = coord_dtype
        
    def _encode_times(self, values):
        if self.attrs and 'calendar' in self.attrs:
            units = self.attrs.get('units')
            calendar = self.attrs.get('calendar')
            
            return xr.coding.times.decode_cf_datetime(
                values, units, calendar
            )
        
        return values
        
    def _create_lookup(self, array):
        # Code by @dcherian
        chunks = array.chunks
        summed = np.cumsum(chunks)
        ends = summed - 1
        starts = np.insert(summed[:-1], 0, 0)

        interleaved = np.vstack([starts, ends]).T.reshape(-1)

        computed, = dask.compute(array[interleaved],)
        known_starts = self._encode_times(computed[::2])
        known_ends = self._encode_times(computed[1::2])
        self._lookup = pd.IntervalIndex.from_tuples(tuple(zip(known_starts, known_ends)), closed="both")
        
        return array
    
    def _replace(self, index, dim=None, coord_dtype=None):
        if dim is None:
            dim = self.dim
        if coord_dtype is None:
            coord_dtype = self.coord_dtype
        return type(self)(index, dim, coord_dtype)
    
    def isel(self, indexers):
        print("calling isel")
        indxr = indexers[self.dim]
        if isinstance(indxr, slice):
            indxr = [indxr.start, indxr.stop]
            
        chunkidx = self._lookup.get_indexer(indxr)
        new_index = self.index.blocks[chunkidx].compute()
        # TODO: Change this so it returns same index??? Not sure how to do this
        return PandasIndex(new_index, dim=self.dim, coord_dtype=self.coord_dtype)
    
    @classmethod
    def from_variables(
        cls,
        variables: Mapping[Any, Variable],
        *,
        options: Mapping[str, Any],
    ) -> "DaskIndex":
        name, var = next(iter(variables.items()))
        dim = var.dims[0]
        attrs = var.attrs
        
        data = var._data
        attrs = var._attrs
        obj = cls(data, dim, coord_dtype=var.dtype, attrs=attrs)
        # obj.index.name = name
        
        return obj
    
    
    

In [169]:
# Create dataset, change the `time` array name to `dim_var1`
# this way, xarray doesn't automatically load the index as Pandas
# @lsetiawan Thoughts: maybe there's a flag when creating dataset like this to not
# auto create index with same name!
ds = xr.Dataset(data_vars=data_vars, coords={'dim_var1': ('time', time.data, time.attrs)})

In [170]:
print(f"Total data size: {memory_repr(ds.nbytes)}")

Total data size: 283.7 GB


In [171]:
new_ds = ds.set_xindex('dim_var1', DaskIndex)

In [172]:
new_ds

<xarray.Dataset>
Dimensions:                      (time: 5252101366)
Coordinates:
  * dim_var1                     (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
Dimensions without coordinates: time
Data variables:
    bottom_pressure              (time) float32 dask.array<chunksize=(13000000,), meta=np.ndarray>
    bottom_pressure_qc_executed  (time) uint8 dask.array<chunksize=(88000000,), meta=np.ndarray>
    bottom_pressure_qc_results   (time) uint8 dask.array<chunksize=(88000000,), meta=np.ndarray>
    deployment                   (time) int32 dask.array<chunksize=(13000000,), meta=np.ndarray>
    driver_timestamp             (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
    ingestion_timestamp          (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
    internal_timestamp           (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
    port_timestamp               (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
    press_trans_temp             (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
Indexes:
    dim_var1                     DaskIndex

In [173]:
new_ds.sel(time=slice("2023-06-01", "2023-06-10"))

calling isel


TypeError: 'str' object cannot be interpreted as an integer